In [1]:
import numpy as np
import pandas as pd

from itertools import islice
from functools import partial
import mxnet as mx
from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters()

from gluonts.dataset.loader import TrainDataLoader
from gluonts.mx import DeepAREstimator
from gluonts.mx.util import get_hybrid_forward_input_names
from gluonts.mx.trainer import Trainer
from gluonts.mx.batchify import batchify
from gluonts.dataset.repository.datasets import get_dataset

pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)

import warnings
warnings.filterwarnings("ignore")

/Users/amritbhaskar/miniconda3/envs/CIS515/lib/python3.8/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [2]:
data = pd.read_csv('walmart_cleaned.csv')
data.head()

,Unnamed: 0,Store,Date,IsHoliday,Dept,Weekly_Sales,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,0,1,2010-02-05,0,1.0,24924.50,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3,151315
1,1,1,2010-02-05,0,26.0,11737.12,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3,151315
2,2,1,2010-02-05,0,17.0,13223.76,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3,151315
3,3,1,2010-02-05,0,45.0,37.44,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3,151315
4,4,1,2010-02-05,0,28.0,1085.29,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3,151315


In [3]:
# data['Store'] = data['Store'].astype(int)
# data['Dept'] = data['Dept'].astype(int)

all_store_dept = data[['Store','Dept']].drop_duplicates()

all_store_dept['combined'] = all_store_dept['Store'].astype(str)+ "_" + all_store_dept['Dept'].astype(str)
list_required_store_dept = all_store_dept["combined"].to_list()

In [4]:
len(list_required_store_dept)

3331

In [5]:
%%time
total_data = []

count=0

for i in list_required_store_dept:
    count+=1
    if(count%100)==0: print(count)
    temp_s, temp_d = i.split("_")
    # print(temp_s, temp_d)
    df_temp = data[(data['Store'].astype(str)==temp_s) & (data['Dept'].astype(str)==temp_d)].copy()
    # print(df_temp[["Date","Weekly_Sales"]].head())
    df_temp.sort_values(['Date'], inplace=True)
    start_date = df_temp["Date"].iloc[0]
    # print(df_temp[["Date","Weekly_Sales"]].head())
    df_temp['Date'] = pd.to_datetime(df_temp['Date'])
    df_temp = df_temp.set_index('Date')
    feat_dynamic_values = df_temp[["IsHoliday","Temperature","Fuel_Price"]].values.T
    target_values =  df_temp["Weekly_Sales"].values
    
    total_data.append({"start": start_date, "target": target_values,
             "feat_dynamic_real": feat_dynamic_values})

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
CPU times: user 15min 34s, sys: 2.69 s, total: 15min 37s
Wall time: 15min 39s


In [6]:
len(total_data)

3331

In [15]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

json_dump = json.dumps({'data' : total_data}, 
                       cls=NumpyEncoder)

In [20]:
import json
with open('deepar_data.json', 'w') as fout:
    json.dump(json_dump, fout)

In [16]:
json_load = json.loads(json_dump)
a_restored = json_load["data"]

In [18]:
len(a_restored)

3331